In [ ]:
!pip install sentence-transformers python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 10.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
from pdfminer.high_level import extract_text
from docx import Document
import torch

# Function to read a PDF document
def read_pdf(file_path):
    return extract_text(file_path)

# Function to read a Word document
def read_word(file_path):
    doc = Document(file_path)
    paragraphs = [paragraph.text for paragraph in doc.paragraphs]
    return " ".join(paragraphs)

# Function to break down the document into chunks
def break_down_document(document):
    # Split the document into paragraphs or chunks based on your preference
    # Ensure proper tokenization
    return [chunk.strip() for chunk in document.split("\n") if chunk.strip()]  # Adjust as needed

# Function to build a semantic similarity search engine
def build_search_engine(chunks):
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

    # Encode chunks to vectors
    chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

    def semantic_search(query, top_k=3):
        query_embedding = model.encode(query, convert_to_tensor=True)

        # Use cosine similarity for semantic search
        cos_sim_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]

        # Get the indices of the top-k similar chunks
        top_results = torch.topk(cos_sim_scores, k=top_k)

        return [chunks[i] for i in top_results.indices]

    return semantic_search

# Main function
def main():
    # Provide the path to your document
    document_path = "/content/JHIMS-1-2.pdf"

    try:
        # Read the document
        if document_path.endswith(".pdf"):
            document_context = read_pdf(document_path)
        elif document_path.endswith(".docx"):
            document_context = read_word(document_path)
        else:
            raise ValueError("Unsupported document format")

        # Break down the document into chunks
        document_chunks = break_down_document(document_context)

        # Build the semantic similarity search engine
        search_engine = build_search_engine(document_chunks)

        # Example usage
        while True:
            user_input = input("User: ")
            if user_input.lower() == 'exit':
                break

            # Get the top-k similar chunks
            similar_chunks = search_engine(user_input)

            if not similar_chunks:
                print("I don't know the answer.")
            else:
                for i, chunk in enumerate(similar_chunks, start=1):
                    print(f"Top-{i} Chunk: {chunk}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


User: what is the metro city
Top-1 Chunk: difference) of the terms that occur in them. The identification number of the cluster and the distance between the
Top-2 Chunk: and to lower costs," in SUGI 31 San Francisco, CA, 2005.
Top-3 Chunk: discharge summaries to extract medication information. The hospital administration is interested to explore the drug
User: what is dog
Top-1 Chunk: National  Library  of  Medicine’s  Unified  Medical  Language  System  (UMLS).    The  RxNorm  vocabulary  is  a
Top-2 Chunk: mild dysplasia/hpv, hsil, + squamose, thinprep, endocervical
Top-3 Chunk: analytical  support  needed  for  the  project.UAB  health  system  has  one  of  the  largest  information  systems  in  the
User: delhi
Top-1 Chunk: and to lower costs," in SUGI 31 San Francisco, CA, 2005.
Top-2 Chunk: 17
Top-3 Chunk: 30
User: who is prasad?
Top-1 Chunk: Uzma Raja
Top-2 Chunk: Uzma Raja, PhD (uraja@cba.ua.edu) is an Assistant Professor at University of Alabama.
Top-3 Chunk: presented here.


KeyboardInterrupt: Interrupted by user